In [66]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
import math

In [78]:
text='''A Boeing 737 MAX aircraft operated by Southwest Airlines made an emergency landing on Tuesday after experiencing an engine problem as it was being ferried from Florida to California, the US Federal Aviation Agency said. "The aircraft returned and landed safely in Orlando," the FAA said in a statement, adding that no passengers were on board the aircraft, which was being transferred to Victorville, California for storage. "The FAA is investigating," added the agency, which grounded the Boeing 737 MAX on March 13 following two deadly accidents involving Ethiopian Airlines and Lion Air but continues to allow the planes to be ferried from airport to airport. Southwest said the plane experienced an engine problem "shortly after takeoff." "The crew followed protocol and safely landed back at the airport" around 3:00 pm (1900 GMT), spokesman Chris Mainz said. "The Boeing 737 MAX 8 will be moved to our Orlando maintenance facility for a review." It was the latest setback for Boeing's flagship narrow-body plane following October's Lion Air crash and the Ethiopian Airlines accident earlier this month, which together killed 346 people. The accidents, which shared similarities, led authorities across the world to ground the aircraft. Boeing has since conducted test flights of its 737 MAX to evaluate a fix for the MCAS stall prevention system targeted as a potential cause for the deadly crashes, two sources familiar with the matter said. The system has been criticized since it can malfunction and make it difficult for pilots to control the aircraft. Both of the recent crashes occurred moments after takeoff. A Senate Commerce Committee panel will hold a hearing Wednesday to question FAA Acting Administration Daniel Elwell and Transportation Department Inspector General Calvin Scovel The officials are expected to face questions from lawmakers on the FAA's certification of the 737 MAX and whether regulators have become too cozy with the company, and fast-tracked some approvals. The session is expected to be followed by a second hearing at a later date with Boeing, airline pilots and other stakeholders, the committee said.'''


In [91]:
def remove_string_special_character(s):
    stripped=re.sub('[^\w\s]','',s)
    stripped=re.sub('_','',stripped)
    stripped=re.sub('\s+',' ',stripped)
    stripped=stripped.strip()
    return stripped

In [92]:
def get_doc(sent):
    doc_info=[]
    i=0
    for sen in sent:
        i+=1
        count=count_words(sen)
        temp={'doc_id':i,'doc_length':count}
        doc_info.append(temp)
    return doc_info

In [93]:
def count_words(sent):
    count=0
    sen=word_tokenize(sent)
    for i in sen:
        count+=1
    return count

In [94]:
def create_freq_dict(sents):
    i=0
    freqDict_list=[]
    for sent in sents:
        i+=1
        freq_dict={}
        words=word_tokenize(sent)
        for word in words:
            word=word.lower()
            if word in freq_dict:
                freq_dict[word]+=1
            else:
                freq_dict[word]=1
            temp={'doc_id':i,'freq_dict':freq_dict}
        freqDict_list.append(temp)
    return freqDict_list

In [95]:
def computeTF(doc_info,freqDict_List):
    TF_scores=[]
    for tempDict in freqDict_list:
        id= tempDict['doc_id']
        for k in tempDict['freq_dict']:
            temp={'doc_id':id,'TF_score':tempDict['freq_dict'][k]/doc_info[id-1]['doc_length'],'key':k}
            TF_scores.append(temp)
        return TF_scores

In [96]:
def computeIDF(doc_info,freqDict_list):
    IDF_scores=[]
    counter=0
    for dict in freqDict_list:
        counter+=1
        for k in dict['freq_dict'].keys():
            count=sum([k in tempDict['freq_dict'] for tempDict in freqDict_list])
            temp={'doc_id':counter,'IDF_score' :math.log(len(doc_info)/count),'key':k}
            IDF_scores.append(temp)
        return IDF_scores

In [97]:
def computeTFIDF(TF_scores,IDF_scores):
    TFIDF_scores=[]
    for j in IDF_scores:
        for i in TF_scores:
            if j['key']==i['key'] and j['doc_id']==i['doc_id']:
                temp={'doc_id':j['doc_id'],'TFIDF_score':j['IDF_score']*i['TF_score'],'key':i['key']}
        TFIDF_scores.append(temp)
    return TFIDF_scores

In [98]:
text_sents=sent_tokenize(text)
sent=[remove_string_special_character(s) for s in text_sents]
print(sent)
doc_info=get_doc(sent)

['A Boeing 737 MAX aircraft operated by Southwest Airlines made an emergency landing on Tuesday after experiencing an engine problem as it was being ferried from Florida to California the US Federal Aviation Agency said', 'The aircraft returned and landed safely in Orlando the FAA said in a statement adding that no passengers were on board the aircraft which was being transferred to Victorville California for storage', 'The FAA is investigating added the agency which grounded the Boeing 737 MAX on March 13 following two deadly accidents involving Ethiopian Airlines and Lion Air but continues to allow the planes to be ferried from airport to airport', 'Southwest said the plane experienced an engine problem shortly after takeoff', 'The crew followed protocol and safely landed back at the airport around 300 pm 1900 GMT spokesman Chris Mainz said', 'The Boeing 737 MAX 8 will be moved to our Orlando maintenance facility for a review', 'It was the latest setback for Boeings flagship narrowbo

In [99]:
freqDict_list=create_freq_dict(sent)
TF_scores=computeTF(doc_info,freqDict_list)
IDF_scores=computeIDF(doc_info,freqDict_list)

In [ ]:
print(TF_scores)

In [ ]:
prit

In [100]:
TFIDF_scores=computeTFIDF(TF_scores,IDF_scores)

In [101]:
def get_sent_score(TFIDF_scores,sent,doc_info):
    sent_info=[]
    for doc in doc_info:
        sent_score=0
        for i in range(0,len(TFIDF_scores)):
            temp_dict=TFIDF_scores[i]
            if doc['doc_id']== temp_dict['doc_id']:
                sent_score+=temp_dict['TFIDF_score']
        temp={'doc_id':doc['doc_id'],'sent_score':sent_score,'sentence': sent[doc['doc_id']-1]}
        sent_info.append(temp)
    return sent_info

In [106]:
sentence_info=get_sent_score(TFIDF_scores,sent,doc_info)

In [103]:
def get_summary(sentence_info):
    sum=0
    summary=[]
    array=[]
    for temp in sentence_info:
        sum+=temp['sent_score']
    avg=sum/len(sentence_info)
    for temp in sentence_info:
        array.append(temp['sent_score'])
    #stdev=statistics.stdev(array)
    for sent in sentence_info:
        if(sent['sent_score'])>=avg:
            summary.append(sent['sentence'])
    summary='\n'.join(summary)
    return summary
    

In [104]:
summary =get_summary(sentence_info)
print(summary)

A Boeing 737 MAX aircraft operated by Southwest Airlines made an emergency landing on Tuesday after experiencing an engine problem as it was being ferried from Florida to California the US Federal Aviation Agency said
